<a href="https://colab.research.google.com/github/fwangliberty/AIoTDesign-Frontend/blob/master/CNN1D_71_features_split_70_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intrusion Detection using CNN for CICIDS 2017 Data Set with 71 Features

We use the pre-processing dataset with data split 70:15:15. In this case, we do not use SMOTE to add more anomalies. 

In [1]:
import os
from os.path import join
import glob
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def display_metrics(y_test, y_pred, label_names):
  print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))

  print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
  print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
  print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

  print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
  print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
  print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

  print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
  print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
  print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

  print('\nClassification Report\n')
  print(classification_report(y_test, y_pred, target_names=label_names))

In [3]:
def display_all(df):
    with pd.option_context("display.max_rows", 100, "display.max_columns", 100): 
        print(df)

In [4]:
def make_value2index(attacks):
    #make dictionary
    attacks = sorted(attacks)
    d = {}
    counter=0
    for attack in attacks:
        d[attack] = counter
        counter+=1
    return d

In [5]:
# chganges label from string to integer/index
def encode_label(Y_str):
    labels_d = make_value2index(np.unique(Y_str))
    Y = [labels_d[y_str] for y_str  in Y_str]
    Y = np.array(Y)
    return np.array(Y)

## Step 1. Loading csv files

In [6]:
# All columns
col_names = ['Source Port', 'Destination Port',
                      'Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets',
                      'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean',
                      'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std',
                      'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total',
                      'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
                      'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Header Length', 'Bwd Header Length',
                      'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
                      'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
                      'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd Segment Size',
                      'Avg Bwd Segment Size','Subflow Fwd Packets', 'Subflow Fwd Bytes',
                      'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward',
                      'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
                      'Idle Std', 'Idle Max', 'Idle Min', 'Label']

### Option 1. Connect to Google Drive

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# load three csv files generated by mlp4nids (Multi-layer perceptron for network intrusion detection )
# first load the train set
df_train = pd.read_csv('/content/drive/My Drive/CICIDS2017/train_set_2.csv',names=col_names, skiprows=1)  

In [9]:
print('Train set size: ', df_train.shape)

Train set size:  (706812, 72)


In [10]:
df_test = pd.read_csv('/content/drive/My Drive/CICIDS2017/test_set_2.csv',names=col_names, skiprows=1)  
print('Test set size: ', df_test.shape)

df_val = pd.read_csv('/content/drive/My Drive/CICIDS2017/crossval_set_2.csv',names=col_names, skiprows=1)  
print('Validation set size: ', df_val.shape)

Test set size:  (314139, 72)
Validation set size:  (235605, 72)


### Option 2. Load from local machine

In [ ]:
dataroot = '../data/cicids2017clean/train_set.csv'
df_train = pd.read_csv(dataroot, names=col_names, skiprows=1) 

In [ ]:
dataroot = '../data/cicids2017clean/crossval_set.csv'
df_val = pd.read_csv(dataroot, names=col_names, skiprows=1) 
dataroot = '../data/cicids2017clean/test_set.csv'
df_test = pd.read_csv(dataroot, names=col_names, skiprows=1) 

## Step 2. Exploring the dataset

In [ ]:
df_train.head()

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
546863,50203,389,17,130,2,2,328,326,164,164,164.0,0.000000,163,163,163.000000,0.000000,5.030769e+06,30769.230770,4.333333e+01,6.812733e+01,122,4,4,4.000000e+00,0.0,4,4,4,4.000,0.000000e+00,4,4,0,0,40,40,15384.615380,15384.615380,163,164,163.600000,0.547723,3.000000e-01,0,0,0,0,0,0,0,0,1,204.500000,164.0,163.000000,2,328,2,326,-1,-1,1,20,0.0,0.0000,0,0,0.0,0.0000,0,0,BENIGN
2284286,47786,80,6,85594954,6,6,360,11595,360,0,60.0,146.969385,4344,0,1932.500000,1754.831473,1.396694e+02,0.140195,7.781359e+06,2.580000e+07,85500000,4,85500000,1.710000e+07,38200000.0,85500000,4,144398,28879.600,5.847893e+04,133148,16,0,0,200,200,0.070098,0.070098,0,4344,919.615385,1498.335273,2.245009e+06,0,0,0,0,1,0,0,0,1,996.250000,60.0,1932.500000,6,360,6,11595,251,235,1,32,495.0,0.0000,495,495,85500000.0,0.0000,85500000,85500000,DoS Hulk
390020,51520,443,6,117915594,19,17,1406,4654,774,0,74.0,179.202245,1418,0,273.764706,485.037567,5.139269e+01,0.305303,3.369017e+06,1.380000e+07,58900000,1,118000000,6.550866e+06,18900000.0,58900000,1,118000000,7366629.625,2.000000e+07,59000000,1,0,0,616,552,0.161132,0.144171,0,1418,163.783784,362.393290,1.313289e+05,0,0,0,1,0,0,0,0,0,168.333333,74.0,273.764706,19,1406,17,4654,29200,181,8,32,495327.0,572653.2552,900254,90400,58400000.0,665473.7481,58900000,58000000,BENIGN
212934,61397,53,17,30731,1,1,46,93,46,46,46.0,0.000000,93,93,93.000000,0.000000,4.523120e+03,65.080863,3.073100e+04,0.000000e+00,30731,30731,0,0.000000e+00,0.0,0,0,0,0.000,0.000000e+00,0,0,0,0,20,20,32.540431,32.540431,46,93,61.666667,27.135463,7.363333e+02,0,0,0,0,0,0,0,0,1,92.500000,46.0,93.000000,1,46,1,93,-1,-1,0,20,0.0,0.0000,0,0,0.0,0.0000,0,0,BENIGN
306030,80,37209,6,49,1,1,0,0,0,0,0.0,0.000000,0,0,0.000000,0.000000,0.000000e+00,40816.326530,4.900000e+01,0.000000e+00,49,49,0,0.000000e+00,0.0,0,0,0,0.000,0.000000e+00,0,0,0,0,32,32,20408.163270,20408.163270,0,0,0.000000,0.000000,0.000000e+00,0,0,0,0,1,1,0,0,1,0.000000,0.0,0.000000,1,0,1,0,332,229,0,32,0.0,0.0000,0,0,0.0,0.0000,0,0,BENIGN


Count the number of attacks

In [ ]:
df_train['Label'].value_counts()

BENIGN                        278274
DoS Hulk                      115062
PortScan                       79402
DDoS                           64012
DoS GoldenEye                   5146
FTP-Patator                     3967
SSH-Patator                     2948
DoS slowloris                   2898
DoS Slowhttptest                2749
Bot                              978
Web Attack � Brute Force         753
Web Attack � XSS                 326
Infiltration                      18
Web Attack � Sql Injection        10
Heartbleed                         5
Name: Label, dtype: int64

In [ ]:
df_train.describe()

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,556548.000000,556548.000000,556548.000000,5.565480e+05,556548.000000,556548.000000,5.565480e+05,5.565480e+05,556548.000000,556548.000000,556548.000000,556548.000000,556548.000000,556548.000000,556548.000000,556548.000000,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,556548.000000,556548.0,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,556548.000000,556548.000000,556548.000000,556548.000000,5.565480e+05,556548.000000,556548.000000,556548.000000,556548.000000,556548.000000,556548.000000,556548.0,556548.000000,556548.000000,556548.000000,556548.000000,556548.000000,556548.000000,5.565480e+05,556548.000000,5.565480e+05,556548.000000,556548.000000,556548.000000,556548.000000,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05
mean,41059.562686,7182.692884,8.358620,2.159931e+07,7.561386,8.259167,6.822941e+02,1.265955e+04,336.406078,18.889621,96.200260,129.272466,1702.530853,25.598678,561.627492,704.125544,9.216068e+05,6.956671e+04,2.010454e+06,5.303410e+06,1.768824e+07,2.166928e+05,2.114477e+07,3.910157e+06,6.895611e+06,1.747174e+07,8.147458e+05,1.071441e+07,1.973938e+06,2.677833e+06,7.299100e+06,6.975517e+05,0.034874,0.0,1.884306e+02,1.994029e+02,6.190488e+04,7.797160e+03,10.577194,1918.313215,308.111075,606.701378,1.260081e+06,0.056247,0.034874,0.000119,0.360393,0.371772,0.093363,0.0,0.000119,0.813482,340.208877,96.200260,561.627492,7.561386,6.822941e+02,8.259167,1.266110e+04,5913.062827,1109.007953,4.883591,25.540766,1.082361e+05,3.242660e+04,1.571849e+05,8.865328e+04,1.640646e+07,1.206008e+06,1.730500e+07,1.552621e+07
std,20762.257722,16999.053361,4.518634,3.824665e+07,641.917381,865.356666,8.207379e+03,1.934124e+06,1243.294768,107.276928,334.915124,524.220046,2851.196636,57.820679,873.686322,1254.293604,2.225586e+07,2.632438e+05,5.270740e+06,1.018751e+07,3.380225e+07,3.879389e+06,3.830840e+07,9.825054e+06,1.406852e+07,3.407836e+07,7.862357e+06,2.923981e+07,8.267067e+06,8.952940e+06,2.272525e+07,7.091812e+06,0.183461,0.0,1.350587e+04,1.735201e+04,2.542560e+05,4.220304e+04,21.863457,2990.143762,434.674128,944.496729,2.616912e+06,0.230398,0.183461,0.010889,0.480115,0.483278,0.290941,0.0,0.010889,1.011007,477.689108,334.915124,873.686322,641.917381,8.207379e+03,865.356666,1.934497e+06,11687.758097,6328.829096,612.739185,6.418384,7.164700e+05,3.567006e+05,9.588739e+05,6.668469e+05,3.283531e+07,7.260774e+06,3.390042e+07,3.259279e+07
min,0.000000,0.000000,0.000000,-1.300000e+01,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.930000e+08,-2.000000e+06,-1.300000e+01,0.000000e+00,-1.300000e+01,-1.300000e+01,0

Read test and validation sets

In [11]:
print('Test set: ')
df_test['Label'].value_counts()

Test set: 


BENIGN                        175063
DoS Hulk                       57443
PortScan                       39575
DDoS                           32261
DoS GoldenEye                   2536
FTP-Patator                     1918
DoS slowloris                   1488
SSH-Patator                     1416
DoS Slowhttptest                1390
Bot                              492
Web Attack � Brute Force         369
Web Attack � XSS                 165
Infiltration                      13
Web Attack � Sql Injection         6
Heartbleed                         4
Name: Label, dtype: int64

In [ ]:
df_test.describe()

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,278270.000000,278270.000000,278270.000000,2.782700e+05,278270.000000,278270.000000,2.782700e+05,2.782700e+05,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,278270.000000,278270.0,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,278270.000000,278270.000000,278270.000000,278270.000000,2.782700e+05,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.0,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,2.782700e+05,278270.000000,2.782700e+05,278270.000000,278270.000000,278270.000000,278270.000000,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05
mean,45364.905750,5840.633640,7.480749,2.018494e+07,13.425781,15.071391,5.018160e+02,2.777075e+04,185.079276,9.858127,42.498337,62.543191,1437.918892,16.702825,472.989076,598.916973,1.319690e+06,9.153090e+04,2.091054e+06,4.928267e+06,1.541155e+07,2.119001e+05,1.981716e+07,4.173283e+06,5.569102e+06,1.522487e+07,1.619357e+06,1.058695e+07,2.418274e+06,2.028897e+06,6.305801e+06,1.360565e+06,0.055008,0.0,3.183946e+02,3.384120e+02,8.541160e+04,6.162424e+03,7.706957,1472.163140,241.332367,466.822599,8.970325e+05,0.058659,0.055008,0.000079,0.423420,0.383936,0.024473,0.0,0.000079,0.515485,266.229446,42.498337,472.989076,13.425781,5.018160e+02,15.071391,2.777075e+04,10570.921080,1731.267470,10.076965,26.450577,9.950258e+04,4.357601e+04,1.635863e+05,7.265777e+04,1.389976e+07,9.310342e+05,1.461183e+07,1.319746e+07
std,17986.051427,15731.041102,3.759347,3.747672e+07,1208.778536,1591.037162,7.898116e+03,3.604963e+06,392.614677,61.235977,99.578231,132.298567,2586.487090,46.983524,797.823718,1144.520735,2.365354e+07,2.939540e+05,5.446991e+06,1.046406e+07,3.156539e+07,3.035605e+06,3.741861e+07,1.178049e+07,1.262896e+07,3.161073e+07,1.066533e+07,2.913323e+07,1.045182e+07,7.731527e+06,2.046630e+07,9.753000e+06,0.227996,0.0,2.524765e+04,3.182466e+04,2.889885e+05,3.511581e+04,17.933308,2584.335069,383.113699,824.111902,2.290032e+06,0.234986,0.227996,0.008891,0.494102,0.486344,0.154512,0.0,0.008891,0.551013,420.649091,99.578231,797.823718,1208.778536,7.898116e+03,1591.037162,3.604963e+06,17760.016938,7434.284715,1153.408495,6.864564,7.423962e+05,4.382250e+05,1.087195e+06,6.664087e+05,3.055265e+07,6.557272e+06,3.158812e+07,3.024680e+07
min,0.000000,0.000000,0.000000,-1.000000e+00,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.200000e+07,-2.000000e+06,-1.000000e+00,0.000000e+00,-1.000000e+00,-1.300000e+

In [ ]:
print('Validation set: ')
df_val['Label'].value_counts()

Validation set: 


BENIGN                        139135
DoS Hulk                       57531
PortScan                       39701
DDoS                           32006
DoS GoldenEye                   2573
FTP-Patator                     1983
SSH-Patator                     1474
DoS slowloris                   1449
DoS Slowhttptest                1374
Bot                              489
Web Attack � Brute Force         376
Web Attack � XSS                 163
Infiltration                       9
Web Attack � Sql Injection         5
Heartbleed                         2
Name: Label, dtype: int64

In [ ]:
df_val.describe()

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,278270.000000,278270.000000,278270.000000,2.782700e+05,278270.000000,278270.000000,278270.000000,2.782700e+05,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,278270.000000,278270.0,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,278270.000000,278270.000000,278270.000000,278270.000000,2.782700e+05,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.0,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,2.782700e+05,278270.000000,278270.000000,278270.000000,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05
mean,42404.657796,4584.284134,8.772361,2.267720e+07,5.697168,5.401071,363.328418,6.993901e+03,146.271053,12.216883,37.118951,48.506259,1644.006185,31.044985,545.020851,677.301293,1.026168e+06,7.647836e+04,2.035292e+06,5.660128e+06,1.867983e+07,1.176209e+05,2.242947e+07,4.097971e+06,7.275898e+06,1.857577e+07,8.532117e+05,1.033244e+07,2.108780e+06,2.497470e+06,6.852268e+06,8.777106e+05,0.032285,0.0,-1.853584e+04,-6.184786e+03,6.922916e+04,7.292921e+03,11.819104,1667.675854,265.849753,521.021331,1.032910e+06,0.063622,0.032285,0.000111,0.325008,0.360617,0.052697,0.0,0.000111,0.635189,293.529747,37.118951,545.020851,5.697168,363.328418,5.401071,6.994360e+03,8300.353962,1196.878449,2.850606,-6.302925e+03,1.011663e+05,3.866528e+04,1.584032e+05,7.772919e+04,1.758377e+07,9.337417e+05,1.829755e+07,1.689405e+07
std,18921.321337,13260.493797,4.780477,3.957518e+07,67.142466,87.166823,3246.913595,2.585565e+05,380.776699,29.589643,80.881268,118.838474,2798.340693,63.404307,851.938455,1229.566919,1.848896e+07,2.733855e+05,4.353752e+06,1.081619e+07,3.532609e+07,1.940212e+06,3.955407e+07,9.992215e+06,1.450108e+07,3.536583e+07,7.979563e+06,2.928691e+07,9.212534e+06,9.080364e+06,2.268132e+07,8.098570e+06,0.176757,0.0,4.255039e+06,8.563325e+05,2.681766e+05,3.540818e+04,21.282926,2802.004249,395.033818,872.649085,2.337799e+06,0.244078,0.176757,0.010554,0.468379,0.480181,0.223428,0.0,0.010554,0.543604,431.308583,80.881268,851.938455,67.142466,3246.913595,87.166823,2.586082e+05,13668.065695,6710.083319,32.991321,7.286969e+05,8.404515e+05,4.487356e+05,1.167019e+06,7.582087e+05,3.455426e+07,6.638199e+06,3.540421e+07,3.437331e+07
min,0.000000,0.000000,0.000000,-1.200000e+01,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.200000e+07,-2.000000e+06,-1.200000e+01,0.000000e+00,-1.200000e+01,-1.200000e+01,0.000000e+

## Step 3. Encode Datasets

Encoding the labels, and generate numpy array. Note that the label has not been encoded as one-hot coding. We will use one-hot code later. 

### Step 3.1 Encoding train dataset

In [12]:
df_label = df_train['Label']
data = df_train.drop(columns=['Label'])
Xtrain = data.values
y_train = encode_label(df_label.values)

### Step 3.2. Encoding test dataset

In [13]:
df_label = df_test['Label']
data = df_test.drop(columns=['Label'])
Xtest = data.values
y_test = encode_label(df_label.values)

### Step 3.3 Encoding validation dataset

In [14]:
df_label = df_val['Label']
data = df_val.drop(columns=['Label'])
Xval = data.values
y_val = encode_label(df_label.values)

## Step 4. Normalization or Standardization

The continuous feature values are normalized into the same feature space. This is important when using features that have different measurements, and is a general requirement of many machine learning algorithms. We implement the two methods to see the impact on the final classifications. 

## Option 1. Normalization

The values of the datasets are normalized using the Min-Max scaling technique, bringing them all within a range of [0,1].

### Step 4.1 Normalizing train dataset

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(Xtrain)
X_train

array([[7.66048676e-01, 5.93603125e-03, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.29167620e-01, 1.22077764e-03, 3.52941176e-01, ...,
        0.00000000e+00, 7.12500000e-01, 7.12500000e-01],
       [7.86144808e-01, 6.76005616e-03, 3.52941176e-01, ...,
        8.96864890e-03, 4.90833333e-01, 4.83333333e-01],
       ...,
       [2.53009842e-01, 1.22077764e-03, 3.52941176e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.85610742e-01, 8.08765183e-04, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.26276036e-01, 1.22077764e-03, 3.52941176e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

### Step 4.2. Normalizing validation dataset

In [ ]:
X_val = scaler.fit_transform(Xval)
X_val

array([[4.37994965e-01, 1.22344737e-03, 3.52941176e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.41290913e-01, 6.12488339e-02, 3.52941176e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.87327382e-02, 6.77483980e-03, 3.52941176e-01, ...,
        5.68691207e-05, 8.33333333e-02, 8.32947583e-02],
       ...,
       [9.30510414e-01, 8.10533882e-04, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.50492103e-01, 8.68494701e-02, 3.52941176e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.24353399e-01, 1.22344737e-03, 3.52941176e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

### Step 4.3. Normalizing test dataset

In [ ]:
X_test = scaler.fit_transform(Xtest)
X_test

array([[5.45487838e-01, 9.66383862e-01, 3.52941176e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.17142247e-01, 1.22074038e-03, 3.52941176e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.92001099e-01, 7.97448653e-02, 3.52941176e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [9.04507584e-01, 1.22074038e-03, 3.52941176e-01, ...,
        0.00000000e+00, 8.33370661e-01, 8.33370661e-01],
       [9.22345653e-01, 8.08740501e-04, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.45030061e-01, 6.75984985e-03, 3.52941176e-01, ...,
        5.01804492e-03, 4.88761059e-01, 4.84412289e-01]])

## Option 2. Standardization

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
scaler = StandardScaler()

X_train = scaler.fit_transform(Xtrain)
X_val = scaler.fit_transform(Xval)
X_test = scaler.fit_transform(Xtest)

X_train

array([[-0.22533952, -0.3324804 , -0.56798023, ..., -0.13401889,
        -0.45860402, -0.43517515],
       [-2.15757735,  2.77557482, -0.56798023, ..., -0.13401889,
        -0.45860402, -0.43517515],
       [ 0.42753629, -0.39535403, -0.56798023, ..., -0.13401889,
        -0.45860402, -0.43517515],
       ...,
       [-0.37188065, -0.36922393, -0.56798023, ..., -0.0447397 ,
         1.37919978,  1.44407525],
       [ 0.24702935, -0.39535403, -0.56798023, ..., -0.13401889,
        -0.45860402, -0.43517515],
       [ 0.10811527,  1.63770485, -0.56798023, ..., -0.13401889,
        -0.45860402, -0.43517515]])

## Step 5 One-hot encoding for labels

y_train, y_test and y_val have to be one-hot-encoded. That means they must have dimension (number_of_samples, 15), where 15 denotes number of classes.

In [17]:
from tensorflow.keras.utils import to_categorical

Save the labels for AdaBoostClassifier

In [18]:
y_train_ada = y_train
y_test_ada = y_test
y_val_ada = y_val

In [19]:
y_train = to_categorical(y_train, 15)
y_test = to_categorical(y_test, 15)
y_val = to_categorical(y_val, 15)

## Step 6. Define the metrics

In [20]:
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier

#importing confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn import metrics
from sklearn.metrics import accuracy_score

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [21]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
]

Get the attacks' names

In [22]:
labels_d = make_value2index(df_test['Label'])

In [23]:
print(labels_d)

{'BENIGN': 175062, 'Bot': 175554, 'DDoS': 207815, 'DoS GoldenEye': 210351, 'DoS Hulk': 267794, 'DoS Slowhttptest': 269184, 'DoS slowloris': 270672, 'FTP-Patator': 272590, 'Heartbleed': 272594, 'Infiltration': 272607, 'PortScan': 312182, 'SSH-Patator': 313598, 'Web Attack � Brute Force': 313967, 'Web Attack � Sql Injection': 313973, 'Web Attack � XSS': 314138}


# CNN1D Model 

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, Flatten, Dense, Activation,Dropout,MaxPooling1D
from tensorflow.keras.constraints import max_norm

In [25]:
#hyper-params
batch_size = 7500 # increasing batch size with more gpu added
input_dim = X_train.shape[1]
num_class = 15                   # 15 intrusion classes, including benign traffic class
num_epochs = 30
learning_rates = 1e-4
regularizations = 1e-3
optim = tf.keras.optimizers.Adam(lr=learning_rates, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

print(input_dim)
print(num_class)

71
15


In [26]:
#X_train_r = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train_r = np.zeros((len(X_train), input_dim, 1))
X_train_r[:, :, 0] = X_train[:, :input_dim]
print(X_train_r.shape)

(706812, 71, 1)


In [27]:
X_test_r = np.zeros((len(X_test), input_dim, 1))
X_test_r[:, :, 0] = X_test[:, :input_dim]
print(X_test_r.shape)

(314139, 71, 1)


In [28]:
X_val_r = np.zeros((len(X_val), input_dim, 1))
X_val_r[:, :, 0] = X_val[:, :input_dim]
print(X_val_r.shape)

(235605, 71, 1)


In [29]:
model = Sequential()

# input layer
model.add(Conv1D(filters=32, kernel_size=17, activation='relu', padding='same', kernel_initializer='he_uniform', input_shape=(71,1)))
model.add(Conv1D(filters=32, kernel_size=11, activation='relu', padding='same', kernel_initializer='he_uniform'))
model.add(MaxPooling1D(pool_size=2,strides=2))
model.add(Dropout(0.3))
model.add(BatchNormalization(axis=1))

model.add(Conv1D(filters=64, kernel_size=17, activation='relu', padding='same', kernel_initializer='he_uniform'))
model.add(Conv1D(filters=64, kernel_size=11, activation='relu', padding='same', kernel_initializer='he_uniform'))
model.add(MaxPooling1D(pool_size=2,strides=2))
model.add(Dropout(0.3))
model.add(BatchNormalization(axis=1)) 

model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_class))
model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 71, 32)            576       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 71, 32)            11296     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 35, 32)            0         
_________________________________________________________________
dropout (Dropout)            (None, 35, 32)            0         
_________________________________________________________________
batch_normalization (BatchNo (None, 35, 32)            140       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 35, 64)            34880     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 35, 64)            4

In [30]:
from keras.optimizers import Nadam
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
import keras
import time
time_start = time.time()

reduce_lr = keras.callbacks.ReduceLROnPlateau(moniter='val_loss',
                                              factor=0.1,
                                              patience=10)
nadam = Nadam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.0001)
model.compile(loss = "categorical_crossentropy",optimizer = "nadam", metrics = METRICS)

history = model.fit(X_train_r, y_train, 
                    epochs=100, 
                    batch_size=batch_size, 
                    verbose=2,
                    validation_data=(X_val_r, y_val),
                    callbacks=[reduce_lr])
time_end = time.time()
train_time = time_end - time_start
print("train_time:",train_time)

Epoch 1/100
95/95 - 39s - loss: 0.5522 - tp: 554669.0000 - fp: 45416.0000 - tn: 9849952.0000 - fn: 152143.0000 - accuracy: 0.9814 - precision: 0.9243 - recall: 0.7847 - auc: 0.9842 - val_loss: 0.2428 - val_tp: 208158.0000 - val_fp: 2970.0000 - val_tn: 3295500.0000 - val_fn: 27447.0000 - val_accuracy: 0.9914 - val_precision: 0.9859 - val_recall: 0.8835 - val_auc: 0.9992
Epoch 2/100
95/95 - 27s - loss: 0.1634 - tp: 662489.0000 - fp: 26628.0000 - tn: 9868740.0000 - fn: 44323.0000 - accuracy: 0.9933 - precision: 0.9614 - recall: 0.9373 - auc: 0.9976 - val_loss: 0.1103 - val_tp: 225146.0000 - val_fp: 5832.0000 - val_tn: 3292638.0000 - val_fn: 10459.0000 - val_accuracy: 0.9954 - val_precision: 0.9748 - val_recall: 0.9556 - val_auc: 0.9995
Epoch 3/100
95/95 - 27s - loss: 0.1187 - tp: 674363.0000 - fp: 21837.0000 - tn: 9873531.0000 - fn: 32449.0000 - accuracy: 0.9949 - precision: 0.9686 - recall: 0.9541 - auc: 0.9986 - val_loss: 0.0825 - val_tp: 228133.0000 - val_fp: 5857.0000 - val_tn: 329261

## Get the metrics

In [31]:
# evaluate model
accuracy = model.evaluate(X_test_r, y_test, batch_size=batch_size, verbose=1)

42/42 [==============================] - 4s 101ms/step - loss: 0.0301 - tp: 310384.0000 - fp: 3250.0000 - tn: 4394696.0000 - fn: 3755.0000 - accuracy: 0.9985 - precision: 0.9896 - recall: 0.9880 - auc: 0.9998


In [32]:
y_pred=model.predict(X_test_r)

In [33]:
display_metrics(y_test_ada, np.argmax(y_pred, axis = 1), labels_d)


Accuracy: 0.99

Micro Precision: 0.99
Micro Recall: 0.99
Micro F1-score: 0.99

Macro Precision: 0.84


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Macro Recall: 0.71
Macro F1-score: 0.72

Weighted Precision: 0.99
Weighted Recall: 0.99
Weighted F1-score: 0.99

Classification Report



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                            precision    recall  f1-score   support

                    BENIGN       1.00      0.99      0.99    175063
                       Bot       0.94      0.99      0.97       492
                      DDoS       1.00      1.00      1.00     32261
             DoS GoldenEye       0.99      0.99      0.99      2536
                  DoS Hulk       0.99      1.00      0.99     57443
          DoS Slowhttptest       0.97      0.99      0.98      1390
             DoS slowloris       0.99      0.99      0.99      1488
               FTP-Patator       1.00      1.00      1.00      1918
                Heartbleed       0.00      0.00      0.00         4
              Infiltration       0.86      0.46      0.60        13
                  PortScan       0.95      0.99      0.97     39575
               SSH-Patator       0.98      0.99      0.99      1416
  Web Attack � Brute Force       0.87      0.17      0.28       369
Web Attack � Sql Injection       0.00      0.00